## FINAL PROJECT


Imports

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re #regular expressions library, 
%matplotlib inline
import pandas as pd
import numpy as np
## plotting libraries
from matplotlib import pyplot as plt
import seaborn as sns
## Sklearn libraries
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline



In [2]:
data = pd.read_csv("TABELA_GERAL_COL7_MAPBIOMAS_DESMAT_VEGSEC_UF.csv") #to predict what category the news belongs
data.head()

,feature_id,city,dr_class_id,dr_class_name,class_id,level_0,level_1,level_2,level_3,level_4,...,2017,2018,2019,2020,2021,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,365644.432700,363193.864600,360503.203200,358538.720700,357198.874600,NaN,NaN,NaN,NaN,NaN
1,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,4,Natural,1. Forest,Savanna Formation,Savanna Formation,Savanna Formation,...,20800.064280,20718.161970,20649.343380,20623.539650,20227.278360,NaN,NaN,NaN,NaN,NaN
2,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,11,Natural,2. Non Forest Natural Formation,Wetland,Wetland,Wetland,...,355.064644,322.257587,276.055484,373.564129,224.389942,NaN,NaN,NaN,NaN,NaN
3,17694,Alta Floresta D'Oeste - RO - AMAZNIA,2,Veg. Primária,12,Natural,2. Non Forest Natural Formation,Grassland,Grassland,Grassland,...,59188.846780,59077.497700,58898.284080,58718.878630,58970.653490,NaN,NaN,NaN,NaN,NaN
4,17694,Alta Floresta D'Oeste - RO - AMAZNIA,3,Veg. Secundária,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,12805.204110,13579.323100,13866.430380,13966.527790,13876.363360,NaN,NaN,NaN,NaN,NaN


#### Examining Data

In [3]:
data.describe()

,feature_id,dr_class_id,class_id,1986,1987,1988,1989,1990,1991,1992,...,2017,2018,2019,2020,2021,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
count,113280.000000,113280.000000,113280.000000,4.198100e+04,7.659000e+04,7.648200e+04,7.631400e+04,7.843300e+04,8.150500e+04,8.391100e+04,...,8.803400e+04,8.814800e+04,8.592800e+04,6.168100e+04,4.477200e+04,0.0,0.0,0.0,0.0,0.0
mean,20773.510373,4.193847,12.707698,2.025679e+04,1.110328e+04,1.111896e+04,1.114344e+04,1.084238e+04,1.043372e+04,1.013455e+04,...,9.659909e+03,9.647416e+03,9.896663e+03,1.378707e+04,1.899402e+04,NaN,NaN,NaN,NaN,NaN
std,1895.433999,1.507697,12.815872,1.922595e+05,1.425423e+05,1.425596e+05,1.426104e+05,1.404663e+05,1.378093e+05,1.356863e+05,...,1.282319e+05,1.279511e+05,1.293485e+05,1.522863e+05,1.784685e+05,NaN,NaN,NaN,NaN,NaN
min,17694.000000,2.000000,0.000000,7.632715e-02,7.518084e-02,7.586495e-02,7.520301e-02,7.471278e-02,7.534580e-02,7.482582e-02,...,7.472381e-02,7.472702e-02,7.579299e-02,7.476924e-02,7.612911e-02,NaN,NaN,NaN,NaN,NaN
25%,19100.000000,3.000000,3.000000,5.934153e+00,4.121333e+00,5.341858e+00,5.726840e+00,5.036193e+00,4.621683e+00,4.288778e+00,...,3.283666e+00,3.398525e+00,3.014386e+00,6.675744e+00,4.392379e+01,NaN,NaN,NaN,NaN,NaN
50%,20722.000000,4.000000,11.000000,1.512885e+02,4.652763e+01,6.361559e+01,6.859900e+01,6.155848e+01,5.449341e+01,4.913048e+01,...,4.445342e+01,4.182031e+01,4.083587e+01,1.395884e+02,7.476427e+02,NaN,NaN,NaN,NaN,NaN
75%,22370.000000,5.000000,15.000000,6.551972e+03,8.188637e+02,9.274477e+02,9.888374e+02,9.575009e+02,8.699481e+02,8.138846e+02,...,9.710906e+02,9.526703e+02,1.037767e+03,2.966953e+03,6.813479e+03,NaN,NaN,NaN,NaN,NaN
max,24232.000000,7.000000,62.000000,1.547810e+07,1.547855e+07,1.547336e+07,1.546564e+07,1.545812e+07,1.545590e+07,1.544646e+07,...,1.474367e+07,1.468377e+07,1.462614e+07,1.454693e+07,1.453010e+07,NaN,NaN,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113280 entries, 0 to 113279
Data columns (total 52 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   feature_id     113280 non-null  int64  
 1   city           113280 non-null  object 
 2   dr_class_id    113280 non-null  int64  
 3   dr_class_name  113280 non-null  object 
 4   class_id       113280 non-null  int64  
 5   level_0        113280 non-null  object 
 6   level_1        113280 non-null  object 
 7   level_2        113280 non-null  object 
 8   level_3        113280 non-null  object 
 9   level_4        113280 non-null  object 
 10  color          112625 non-null  object 
 11  1986           41981 non-null   float64
 12  1987           76590 non-null   float64
 13  1988           76482 non-null   float64
 14  1989           76314 non-null   float64
 15  1990           78433 non-null   float64
 16  1991           81505 non-null   float64
 17  1992           83911 non-null

#### Dropping Columns

In [5]:
data = data.drop(['Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50','Unnamed: 51'], axis = 1)
data.tail()

,feature_id,city,dr_class_id,dr_class_name,class_id,level_0,level_1,level_2,level_3,level_4,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
113275,24232,Santo Antnio do Leverger - MT - PANTANAL,6,Supressão Veg. Secundária,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,19.516393,13.693674,46.355575,58.292902,93.945790,38.174683,63.411254,109.230359,81.341857,NaN
113276,24232,Santo Antnio do Leverger - MT - PANTANAL,6,Supressão Veg. Secundária,4,Natural,1. Forest,Savanna Formation,Savanna Formation,Savanna Formation,...,237.350385,143.010998,606.715422,423.452301,540.467627,253.237186,449.122907,439.661795,270.674854,NaN
113277,24232,Santo Antnio do Leverger - MT - PANTANAL,6,Supressão Veg. Secundária,11,Natural,2. Non Forest Natural Formation,Wetland,Wetland,Wetland,...,85.838827,51.755217,8.319485,337.080681,117.804520,154.690891,86.661810,200.380994,2123.548449,NaN
113278,24232,Santo Antnio do Leverger - MT - PANTANAL,6,Supressão Veg. Secundária,12,Natural,2. Non Forest Natural Formation,Grassland,Grassland,Grassland,...,200.934955,426.996352,608.960854,917.410477,826.044276,453.802516,676.725529,352.502942,1192.245191,NaN
113279,24232,Santo Antnio do Leverger - MT - PANTANAL,7,Outras transições,0,Not applied,6. Non Observed,Non Observed,Non Observed,Non Observed,...,253745.118600,256760.233800,256588.734100,258328.342900,256337.628500,258461.312900,259995.589000,260751.460800,265211.185500,276266.7333


In [6]:
data = data.drop(['level_0', 'level_1', 'level_2', 'level_3','level_4','color','feature_id','dr_class_id','class_id','1986','1987','1988','1989','1990','1991','1992','1993','1994','1995','1996','1997','1998'], axis = 1)
data = data.reset_index(drop=True)
data.tail()

,city,dr_class_name,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
113275,Santo Antnio do Leverger - MT - PANTANAL,Supressão Veg. Secundária,11.211184,28.968882,16.355516,49.943623,23.650003,30.408017,14.812768,12.708954,...,19.516393,13.693674,46.355575,58.292902,93.945790,38.174683,63.411254,109.230359,81.341857,NaN
113276,Santo Antnio do Leverger - MT - PANTANAL,Supressão Veg. Secundária,314.149530,328.685630,109.145684,350.743366,456.578762,287.321731,384.056114,199.172857,...,237.350385,143.010998,606.715422,423.452301,540.467627,253.237186,449.122907,439.661795,270.674854,NaN
113277,Santo Antnio do Leverger - MT - PANTANAL,Supressão Veg. Secundária,110.925312,68.570020,38.560021,5.585289,95.389252,2.152516,92.640119,59.759220,...,85.838827,51.755217,8.319485,337.080681,117.804520,154.690891,86.661810,200.380994,2123.548449,NaN
113278,Santo Antnio do Leverger - MT - PANTANAL,Supressão Veg. Secundária,899.071988,727.498942,147.151579,307.222916,404.635390,235.034512,386.067468,422.626773,...,200.934955,426.996352,608.960854,917.410477,826.044276,453.802516,676.725529,352.502942,1192.245191,NaN
113279,Santo Antnio do Leverger - MT - PANTANAL,Outras transições,179027.276700,187466.902000,199579.976600,204422.093800,211007.350400,219264.340400,226551.809400,238883.155800,...,253745.118600,256760.233800,256588.734100,258328.342900,256337.628500,258461.312900,259995.589000,260751.460800,265211.185500,276266.7333


#### Dropping Rows/Biomes

In [7]:
data = data[:40606]
data

,city,dr_class_name,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,434860.595600,426677.757300,419579.412800,410698.915300,399517.735300,392929.590500,386405.624300,382333.454200,...,372863.430900,371629.571400,371149.393100,370178.330500,367974.498500,365644.432700,363193.864600,360503.203200,358538.720700,357198.874600
1,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,23853.322810,23527.521100,23060.400780,22695.549600,22425.250660,22120.956000,21708.615150,21454.202420,...,21252.493680,21284.406160,21118.946160,21016.959810,20901.929730,20800.064280,20718.161970,20649.343380,20623.539650,20227.278360
2,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,596.460339,1634.673966,160.501218,173.453450,200.396530,377.613166,544.274872,508.637273,...,366.693819,423.736574,400.859707,300.688970,312.384479,355.064644,322.257587,276.055484,373.564129,224.389942
3,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,67383.223400,65427.993720,65504.703240,64426.362940,63376.054480,62283.167760,61623.861420,61212.092160,...,59718.293740,59534.674670,59487.559910,59495.260190,59343.922070,59188.846780,59077.497700,58898.284080,58718.878630,58970.653490
4,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Secundária,5186.465217,5022.483462,4667.280839,4359.029923,4400.882147,4946.424254,5913.960331,6716.013413,...,9946.651946,10538.813840,11722.803430,12488.906800,12420.973660,12805.204110,13579.323100,13866.430380,13966.527790,13876.363360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40601,Braslia - DF - CERRADO,Supressão Veg. Secundária,10.763938,10.162166,28.076535,15.676177,27.025726,16.010241,12.225429,12.053547,...,16.362187,21.268472,37.703440,49.144290,38.042205,34.171281,21.426125,40.624733,13.340106,NaN
40602,Braslia - DF - CERRADO,Supressão Veg. Secundária,112.094661,134.656728,45.051596,83.095813,152.612834,71.984427,41.589522,63.250461,...,18.772872,70.906405,92.279196,86.331891,42.377472,40.046839,29.806186,31.097545,47.634865,NaN
40603,Braslia - DF - CERRADO,Supressão Veg. Secundária,5.250349,4.389324,3.531846,3.271210,2.840724,3.357783,1.549436,3.271980,...,3.012664,2.753466,1.549897,2.839321,1.033177,6.367129,5.076440,2.667502,0.172337,NaN
40604,Braslia - DF - CERRADO,Supressão Veg. Secundária,191.408217,348.135098,264.962524,288.578473,205.894130,166.140877,154.901631,116.761409,...,106.404916,159.904133,196.078674,115.826068,113.596762,121.757033,146.114612,107.630835,61.215093,NaN


#### Data Cleaning

:
Null Values

In [8]:
threshold = 0.65
data = data.dropna(thresh=len(data)* (1-threshold), axis=1)
data

,city,dr_class_name,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,434860.595600,426677.757300,419579.412800,410698.915300,399517.735300,392929.590500,386405.624300,382333.454200,...,372863.430900,371629.571400,371149.393100,370178.330500,367974.498500,365644.432700,363193.864600,360503.203200,358538.720700,357198.874600
1,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,23853.322810,23527.521100,23060.400780,22695.549600,22425.250660,22120.956000,21708.615150,21454.202420,...,21252.493680,21284.406160,21118.946160,21016.959810,20901.929730,20800.064280,20718.161970,20649.343380,20623.539650,20227.278360
2,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,596.460339,1634.673966,160.501218,173.453450,200.396530,377.613166,544.274872,508.637273,...,366.693819,423.736574,400.859707,300.688970,312.384479,355.064644,322.257587,276.055484,373.564129,224.389942
3,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,67383.223400,65427.993720,65504.703240,64426.362940,63376.054480,62283.167760,61623.861420,61212.092160,...,59718.293740,59534.674670,59487.559910,59495.260190,59343.922070,59188.846780,59077.497700,58898.284080,58718.878630,58970.653490
4,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Secundária,5186.465217,5022.483462,4667.280839,4359.029923,4400.882147,4946.424254,5913.960331,6716.013413,...,9946.651946,10538.813840,11722.803430,12488.906800,12420.973660,12805.204110,13579.323100,13866.430380,13966.527790,13876.363360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40601,Braslia - DF - CERRADO,Supressão Veg. Secundária,10.763938,10.162166,28.076535,15.676177,27.025726,16.010241,12.225429,12.053547,...,16.362187,21.268472,37.703440,49.144290,38.042205,34.171281,21.426125,40.624733,13.340106,NaN
40602,Braslia - DF - CERRADO,Supressão Veg. Secundária,112.094661,134.656728,45.051596,83.095813,152.612834,71.984427,41.589522,63.250461,...,18.772872,70.906405,92.279196,86.331891,42.377472,40.046839,29.806186,31.097545,47.634865,NaN
40603,Braslia - DF - CERRADO,Supressão Veg. Secundária,5.250349,4.389324,3.531846,3.271210,2.840724,3.357783,1.549436,3.271980,...,3.012664,2.753466,1.549897,2.839321,1.033177,6.367129,5.076440,2.667502,0.172337,NaN
40604,Braslia - DF - CERRADO,Supressão Veg. Secundária,191.408217,348.135098,264.962524,288.578473,205.894130,166.140877,154.901631,116.761409,...,106.404916,159.904133,196.078674,115.826068,113.596762,121.757033,146.114612,107.630835,61.215093,NaN


In [9]:
data.isnull().sum()

city                 0
dr_class_name        0
1999              9151
2000              9213
2001              9355
2002              9245
2003              8838
2004              8954
2005              8986
2006              9371
2007              8997
2008              8780
2009              9014
2010              8742
2011              8703
2012              8159
2013              7875
2014              8215
2015              7896
2016              7970
2017              7869
2018              7552
2019              7968
2020             15838
2021             23622
dtype: int64

In [10]:
from skimpy import clean_columns
from nltk.metrics import edit_distance
   
def fill_nans_with_means(df):
    return df.fillna(df.mean()).round()

x = fill_nans_with_means(data)
x

,city,dr_class_name,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,434861.0,426678.0,419579.0,410699.0,399518.0,392930.0,386406.0,382333.0,...,372863.0,371630.0,371149.0,370178.0,367974.0,365644.0,363194.0,360503.0,358539.0,357199.0
1,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,23853.0,23528.0,23060.0,22696.0,22425.0,22121.0,21709.0,21454.0,...,21252.0,21284.0,21119.0,21017.0,20902.0,20800.0,20718.0,20649.0,20624.0,20227.0
2,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,596.0,1635.0,161.0,173.0,200.0,378.0,544.0,509.0,...,367.0,424.0,401.0,301.0,312.0,355.0,322.0,276.0,374.0,224.0
3,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Primária,67383.0,65428.0,65505.0,64426.0,63376.0,62283.0,61624.0,61212.0,...,59718.0,59535.0,59488.0,59495.0,59344.0,59189.0,59077.0,58898.0,58719.0,58971.0
4,Alta Floresta D'Oeste - RO - AMAZNIA,Veg. Secundária,5186.0,5022.0,4667.0,4359.0,4401.0,4946.0,5914.0,6716.0,...,9947.0,10539.0,11723.0,12489.0,12421.0,12805.0,13579.0,13866.0,13967.0,13876.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40601,Braslia - DF - CERRADO,Supressão Veg. Secundária,11.0,10.0,28.0,16.0,27.0,16.0,12.0,12.0,...,16.0,21.0,38.0,49.0,38.0,34.0,21.0,41.0,13.0,36446.0
40602,Braslia - DF - CERRADO,Supressão Veg. Secundária,112.0,135.0,45.0,83.0,153.0,72.0,42.0,63.0,...,19.0,71.0,92.0,86.0,42.0,40.0,30.0,31.0,48.0,36446.0
40603,Braslia - DF - CERRADO,Supressão Veg. Secundária,5.0,4.0,4.0,3.0,3.0,3.0,2.0,3.0,...,3.0,3.0,2.0,3.0,1.0,6.0,5.0,3.0,0.0,36446.0
40604,Braslia - DF - CERRADO,Supressão Veg. Secundária,191.0,348.0,265.0,289.0,206.0,166.0,155.0,117.0,...,106.0,160.0,196.0,116.0,114.0,122.0,146.0,108.0,61.0,36446.0


clnr = Cleaner(data)

clnr.cdf

In [11]:
#x = x.astype(str).replace(r'\.0$', '', regex=True)
#x

Splitting City Column

In [12]:
ge = x['city'].str.split(' - ', expand=True)
ge = ge.rename(columns={0: 'City', 2: 'Bioma'})
x['city'] = ge['City']
x = x.join(ge['Bioma'])

In [13]:
x

,city,dr_class_name,1999,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Bioma
0,Alta Floresta D'Oeste,Veg. Primária,434861.0,426678.0,419579.0,410699.0,399518.0,392930.0,386406.0,382333.0,...,371630.0,371149.0,370178.0,367974.0,365644.0,363194.0,360503.0,358539.0,357199.0,AMAZNIA
1,Alta Floresta D'Oeste,Veg. Primária,23853.0,23528.0,23060.0,22696.0,22425.0,22121.0,21709.0,21454.0,...,21284.0,21119.0,21017.0,20902.0,20800.0,20718.0,20649.0,20624.0,20227.0,AMAZNIA
2,Alta Floresta D'Oeste,Veg. Primária,596.0,1635.0,161.0,173.0,200.0,378.0,544.0,509.0,...,424.0,401.0,301.0,312.0,355.0,322.0,276.0,374.0,224.0,AMAZNIA
3,Alta Floresta D'Oeste,Veg. Primária,67383.0,65428.0,65505.0,64426.0,63376.0,62283.0,61624.0,61212.0,...,59535.0,59488.0,59495.0,59344.0,59189.0,59077.0,58898.0,58719.0,58971.0,AMAZNIA
4,Alta Floresta D'Oeste,Veg. Secundária,5186.0,5022.0,4667.0,4359.0,4401.0,4946.0,5914.0,6716.0,...,10539.0,11723.0,12489.0,12421.0,12805.0,13579.0,13866.0,13967.0,13876.0,AMAZNIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40601,Braslia,Supressão Veg. Secundária,11.0,10.0,28.0,16.0,27.0,16.0,12.0,12.0,...,21.0,38.0,49.0,38.0,34.0,21.0,41.0,13.0,36446.0,CERRADO
40602,Braslia,Supressão Veg. Secundária,112.0,135.0,45.0,83.0,153.0,72.0,42.0,63.0,...,71.0,92.0,86.0,42.0,40.0,30.0,31.0,48.0,36446.0,CERRADO
40603,Braslia,Supressão Veg. Secundária,5.0,4.0,4.0,3.0,3.0,3.0,2.0,3.0,...,3.0,2.0,3.0,1.0,6.0,5.0,3.0,0.0,36446.0,CERRADO
40604,Braslia,Supressão Veg. Secundária,191.0,348.0,265.0,289.0,206.0,166.0,155.0,117.0,...,160.0,196.0,116.0,114.0,122.0,146.0,108.0,61.0,36446.0,CERRADO


In [14]:
x.rename(columns={'dr_class_name':'Deforestation'}, inplace=True)
x

,city,Deforestation,1999,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Bioma
0,Alta Floresta D'Oeste,Veg. Primária,434861.0,426678.0,419579.0,410699.0,399518.0,392930.0,386406.0,382333.0,...,371630.0,371149.0,370178.0,367974.0,365644.0,363194.0,360503.0,358539.0,357199.0,AMAZNIA
1,Alta Floresta D'Oeste,Veg. Primária,23853.0,23528.0,23060.0,22696.0,22425.0,22121.0,21709.0,21454.0,...,21284.0,21119.0,21017.0,20902.0,20800.0,20718.0,20649.0,20624.0,20227.0,AMAZNIA
2,Alta Floresta D'Oeste,Veg. Primária,596.0,1635.0,161.0,173.0,200.0,378.0,544.0,509.0,...,424.0,401.0,301.0,312.0,355.0,322.0,276.0,374.0,224.0,AMAZNIA
3,Alta Floresta D'Oeste,Veg. Primária,67383.0,65428.0,65505.0,64426.0,63376.0,62283.0,61624.0,61212.0,...,59535.0,59488.0,59495.0,59344.0,59189.0,59077.0,58898.0,58719.0,58971.0,AMAZNIA
4,Alta Floresta D'Oeste,Veg. Secundária,5186.0,5022.0,4667.0,4359.0,4401.0,4946.0,5914.0,6716.0,...,10539.0,11723.0,12489.0,12421.0,12805.0,13579.0,13866.0,13967.0,13876.0,AMAZNIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40601,Braslia,Supressão Veg. Secundária,11.0,10.0,28.0,16.0,27.0,16.0,12.0,12.0,...,21.0,38.0,49.0,38.0,34.0,21.0,41.0,13.0,36446.0,CERRADO
40602,Braslia,Supressão Veg. Secundária,112.0,135.0,45.0,83.0,153.0,72.0,42.0,63.0,...,71.0,92.0,86.0,42.0,40.0,30.0,31.0,48.0,36446.0,CERRADO
40603,Braslia,Supressão Veg. Secundária,5.0,4.0,4.0,3.0,3.0,3.0,2.0,3.0,...,3.0,2.0,3.0,1.0,6.0,5.0,3.0,0.0,36446.0,CERRADO
40604,Braslia,Supressão Veg. Secundária,191.0,348.0,265.0,289.0,206.0,166.0,155.0,117.0,...,160.0,196.0,116.0,114.0,122.0,146.0,108.0,61.0,36446.0,CERRADO


In [15]:
#hot encode then can do this
#data = data[~data['dr_class_name'] != Veg.Secundaria]

In [16]:
x = x[~x.isin(['Veg. Secundária', 'Supressão Veg. Primária','Recuperação para Veg. Secundária','Supressão Veg. Secundária','Outras transições']).any(axis=1)]
x

,city,Deforestation,1999,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Bioma
0,Alta Floresta D'Oeste,Veg. Primária,434861.0,426678.0,419579.0,410699.0,399518.0,392930.0,386406.0,382333.0,...,371630.0,371149.0,370178.0,367974.0,365644.0,363194.0,360503.0,358539.0,357199.0,AMAZNIA
1,Alta Floresta D'Oeste,Veg. Primária,23853.0,23528.0,23060.0,22696.0,22425.0,22121.0,21709.0,21454.0,...,21284.0,21119.0,21017.0,20902.0,20800.0,20718.0,20649.0,20624.0,20227.0,AMAZNIA
2,Alta Floresta D'Oeste,Veg. Primária,596.0,1635.0,161.0,173.0,200.0,378.0,544.0,509.0,...,424.0,401.0,301.0,312.0,355.0,322.0,276.0,374.0,224.0,AMAZNIA
3,Alta Floresta D'Oeste,Veg. Primária,67383.0,65428.0,65505.0,64426.0,63376.0,62283.0,61624.0,61212.0,...,59535.0,59488.0,59495.0,59344.0,59189.0,59077.0,58898.0,58719.0,58971.0,AMAZNIA
19,Ariquemes,Veg. Primária,206201.0,198623.0,190982.0,180079.0,168211.0,161475.0,152928.0,148623.0,...,139452.0,137433.0,134671.0,131403.0,128311.0,125044.0,122515.0,120027.0,119196.0,AMAZNIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40561,Vila Propcio,Veg. Primária,1370.0,1370.0,1384.0,1382.0,1354.0,1329.0,1319.0,1309.0,...,1295.0,1295.0,1294.0,1294.0,1278.0,1273.0,1268.0,1267.0,1267.0,CERRADO
40582,Braslia,Veg. Primária,40867.0,40847.0,40652.0,40417.0,40483.0,40570.0,40709.0,40590.0,...,40761.0,40734.0,40696.0,40693.0,40639.0,40725.0,40702.0,40709.0,40629.0,CERRADO
40583,Braslia,Veg. Primária,122345.0,121007.0,120021.0,119149.0,118292.0,117585.0,116839.0,116461.0,...,114203.0,113702.0,113083.0,112658.0,112175.0,111690.0,111356.0,111065.0,110900.0,CERRADO
40584,Braslia,Veg. Primária,3044.0,2944.0,2970.0,3063.0,3027.0,2940.0,2978.0,2982.0,...,2829.0,2904.0,2937.0,2948.0,2945.0,2916.0,2908.0,2938.0,3044.0,CERRADO


In [17]:
len(x['Deforestation'].unique())

1

In [18]:
x = x.groupby('city', as_index=False).sum()
x

,city,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aailndia,132743.0,126037.0,121485.0,117435.0,107984.0,100747.0,94523.0,85893.0,80325.0,...,66942.0,65806.0,65177.0,64052.0,63043.0,61533.0,60188.0,58964.0,57544.0,56941.0
1,Abadia de Gois,1331.0,1278.0,1259.0,1240.0,1223.0,1217.0,1207.0,1195.0,1181.0,...,1145.0,1140.0,1134.0,1132.0,1132.0,1130.0,1117.0,1110.0,1105.0,1099.0
2,Abadia dos Dourados,14181.0,14034.0,13916.0,13741.0,13590.0,13311.0,13111.0,12950.0,12807.0,...,12400.0,12363.0,12301.0,12195.0,12144.0,12063.0,11997.0,11964.0,11945.0,11906.0
3,Abadinia,22677.0,22293.0,21930.0,21590.0,21118.0,20425.0,19907.0,19668.0,19505.0,...,18674.0,18520.0,18363.0,18213.0,18130.0,17935.0,17727.0,17592.0,17498.0,17425.0
4,Abaet,43329.0,42542.0,41444.0,40546.0,39914.0,39398.0,38609.0,37824.0,37051.0,...,33727.0,33073.0,32573.0,32053.0,31763.0,31366.0,30907.0,30458.0,30220.0,30100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,guas Lindas de Gois,7460.0,7352.0,7262.0,7113.0,6995.0,6909.0,6833.0,6734.0,6649.0,...,6305.0,6261.0,6188.0,6088.0,6009.0,5916.0,5833.0,5780.0,5736.0,5723.0
1833,guas de Santa Brbara,5186.0,5169.0,5039.0,4969.0,4951.0,4919.0,4889.0,4809.0,4794.0,...,4715.0,4711.0,4696.0,4675.0,4654.0,4641.0,4625.0,4610.0,4584.0,4578.0
1834,guas de So Pedro,65.0,65.0,65.0,65.0,65.0,65.0,65.0,64.0,65.0,...,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0
1835,leo,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [19]:
pd.options.display.precision = 0
x

,city,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aailndia,132743,126037,121485,117435,107984,100747,94523,85893,80325,...,66942,65806,65177,64052,63043,61533,60188,58964,57544,56941
1,Abadia de Gois,1331,1278,1259,1240,1223,1217,1207,1195,1181,...,1145,1140,1134,1132,1132,1130,1117,1110,1105,1099
2,Abadia dos Dourados,14181,14034,13916,13741,13590,13311,13111,12950,12807,...,12400,12363,12301,12195,12144,12063,11997,11964,11945,11906
3,Abadinia,22677,22293,21930,21590,21118,20425,19907,19668,19505,...,18674,18520,18363,18213,18130,17935,17727,17592,17498,17425
4,Abaet,43329,42542,41444,40546,39914,39398,38609,37824,37051,...,33727,33073,32573,32053,31763,31366,30907,30458,30220,30100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,guas Lindas de Gois,7460,7352,7262,7113,6995,6909,6833,6734,6649,...,6305,6261,6188,6088,6009,5916,5833,5780,5736,5723
1833,guas de Santa Brbara,5186,5169,5039,4969,4951,4919,4889,4809,4794,...,4715,4711,4696,4675,4654,4641,4625,4610,4584,4578
1834,guas de So Pedro,65,65,65,65,65,65,65,64,65,...,64,64,64,64,64,64,64,64,64,64
1835,leo,3,3,3,3,3,3,3,3,3,...,3,2,2,2,2,2,2,2,2,2


In [ ]:
data1 = pd.read_csv("TABELA_GERAL_COL7_MAPBIOMAS_DESMAT_VEGSEC_UF.csv") #to predict what category the news belongs
data1.head()